# 注册会计师带你探索风险分析（EDA）
# 项目介绍
所谓探索性数据分析(Exploratory Data Analysis，以下简称EDA)，是指对已有的数据(特别是调查或观察得来的原始数据)在尽量少的先验假定下进行探索，通过作图、制表、方程拟合、计算特征量等手段探索数据的结构和规律的一种数据分析方法。特别是当我们对这些数据中的信息没有足够的经验，不知道该用何种传统统计方法进行分析时，探索性数据分析就会非常有效。探索性数据分析在上世纪六十年代被提出，其方法由美国著名统计学家约翰·图基(John Tukey)命名。

# 本项目需解决的问题
本项目分析P2P平台Lending Club的贷款数据，探索数据分析过程中，并尝试回答以下3个问题：

* 利率与风险成正比，风险越高，利率越高，违约的可能线性越大，从P2P平台的数据来看，影响风险的因素有哪些？（为后续建模做准备）
* 了解P2P平台的业务特点、产品类型、资产质量、风险定价？
* 有什么建议？

# 分析思路
我们可以将信贷信息分为信贷硬信息和信贷软信息。

任何可以量化客户的还款能力的信息均可以用作硬信息，可勾勒客户还款意愿的信息则为软信息。

信贷硬信息： 站在企业的角度，硬信息主要包括财务三大报表（资产负债表、利润表和现金流量表）以及信贷记录；站在个人角度硬信息主要包括：个人年收入 、资产状况（借款是否拥有房产、车或理财产品）。

信贷软信息： 过往的信贷记录比较直接了解客户的还款意愿，以往发生违约次数较多的客户再次发生违约的概率相比其他客户大。客户的学历、年龄、目前工作所在单位的级别和性别等信息也可作为软信息。

因此，我们主要围绕着“客户是否具有偿还能力，是否具有偿还意愿”展开探索分析。

# 项目背景
作为旧金山的一家个人对个人的借贷公司，Lending Club成立于2006年。他们是第一家注册为按照美国证券交易委员会SEC(Securities and Exchange Commission)的安全标准向个人提供个人贷款的借贷公司。与传统借贷机构最大的不同是，Lending Club利用网络技术打造的这个交易平台，直接连接了个人投资者和个人借贷者，通过此种方式，缩短了资金流通的环节，尤其是绕过了传统的大银行等金融机构，使得投资者和借贷者都能得到更多实惠、更快捷。对于投资者来说可以获得更好的回报，而对于借贷者来说，则可以获得相对较低的贷款利率。
<br>Lending Club 介绍[《来认识一下即将上市的全球最大P2P网贷公司Lending Club》](http://link.zhihu.com/?target=https%3A//www.huxiu.com/article/41472/1.html)
数据集

数据集是Lending Club平台发生借贷的业务数据（2017年第二季），具体数据集可以从[Lending Club官网下载](http://link.zhihu.com/?target=https%3A//www.lendingclub.com/info/download-data.action)

本项目报告分析，我将如何运用Python操作数据和探索分析数据的思考过程均记录下来。

# 前期准备
第一步，导入需要的包

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')  #风格设置近似R这种的ggplot库
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
from pyecharts import Pie
import warnings # 忽略warnings
warnings.filterwarnings('ignore')
# mac 中指定默认字体
plt.rcParams['font.sans-serif']=['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False #解决保存图像是负号'-'显示为方块的问题

注意：不要漏了%matplotlib inline。IPython提供了很多魔法命令，使得在IPython环境中的操作更加得心应手，使用%matplotlib inline在绘图时，将图片内嵌在交互窗口，而不是弹出一个图片窗口。具体请查看的[Stackoverflow](http://link.zhihu.com/?target=https%3A//stackoverflow.com/questions/19410042/how-to-make-ipython-notebook-matplotlib-plot-inline)解释。

# 获取数据
## 使用pandas解析数据
Pandas是基于NumPy的一个非常好用的库，无论是读取数据、处理数据，用它都非常简单。学习Pandas最好的方法就是查看官方文档 。
数据集的格式是CSV，因此我们用到pandas.read_csv方法，同时也将CSV内容转化成矩阵的格式。

In [28]:
data = pd.read_csv("LoanStats_2017Q2.csv",encoding='latin-1',skiprows=1)

# 探索分析数据（EDA）

一旦获得了数据，下一步就是检查和探索他们。在这个阶段，主要的目标是合理地检查数据。例如：如果数据有唯一的标记符，是否真的只有一个；数据是什么类型，检查最极端的情况。他们是否有意义，有什么需要删除的吗？数据应该怎么调整才能适用于接下来的分析和挖掘？此外，数据集还有可能存在异常值。同时，我将会通过对数据进行简单的统计测试，并将其可视化。 检查和探索数据的过程非常关键。因为下一步需要清洗和准备处理这些数据，只有进入模型的数据质量是好的，才能构建好的模型。（避免Garbage in, Garbage out）

首先预览基本内容，Pandas为我们提供很多可以方便查看和检查数数据的方法，有df.head(n)、df.tail(n)、df.shape()、[df.info()](http://link.zhihu.com/?target=http%3A//df.info%28%29) 等 。

查看表格的行数和列数

In [29]:
data.shape

(105453, 145)

In [30]:
data.head()#显示前五行

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,25000.0,25000.0,25000.0,60 months,15.05%,595.41,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,26000.0,26000.0,26000.0,36 months,9.93%,838.10,B,B2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,20000.0,20000.0,20000.0,36 months,9.44%,640.10,B,B1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,10000.0,10000.0,10000.0,36 months,7.21%,309.74,A,A3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,15000.0,15000.0,15000.0,36 months,7.97%,469.84,A,A5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


扩大行查看范围


In [31]:
data.iloc[0]

id                                                                NaN
member_id                                                         NaN
loan_amnt                                                       25000
funded_amnt                                                     25000
funded_amnt_inv                                                 25000
term                                                        60 months
int_rate                                                       15.05%
installment                                                    595.41
grade                                                               C
sub_grade                                                          C4
emp_title                                                  Supervisor
emp_length                                                  10+ years
home_ownership                                                   RENT
annual_inc                                                      60000
verification_status 

由上图我们发现一些列有很多缺失值，这些缺失值对我们的数据分析没有意义，因此，首先把含有许多缺失值的列删除，同时将已清洗过的数据新建CSV保存。

### 处理缺失值
统计每列属性缺失值的数量。

In [32]:
def not_null_count(column):
    column_null = pd.isnull(column)  #判断某列属性是否存在缺失值
    null = column[column_null]
    return len(null)
column_null_count = data.apply(not_null_count)
print(column_null_count)

id                                            105451
member_id                                     105453
loan_amnt                                          2
funded_amnt                                        2
funded_amnt_inv                                    2
term                                               2
int_rate                                           2
installment                                        2
grade                                              2
sub_grade                                          2
emp_title                                       6784
emp_length                                         2
home_ownership                                     2
annual_inc                                         2
verification_status                                2
issue_d                                            2
loan_status                                        2
pymnt_plan                                         2
url                                           

In [34]:
half_count =len(data)/2 #设定阀值
data = data.dropna(thresh=half_count,axis=1)#若某一列数据缺失的数量超过阀值就会被删除
#data = data.drop(['desc', 'url'], axis = 1) # 删除某些加载了网址的url 和 描述的列
data.to_csv('loans_2017q2.csv',index=False) # 将预处理后的数据转化为csv

(105453, 103)


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,25000.0,25000.0,25000.0,60 months,15.05%,595.41,C,C4,Supervisor,10+ years,...,57.1,0.0,0.0,36800.0,26947.0,24800.0,0.0,N,Cash,N
1,26000.0,26000.0,26000.0,36 months,9.93%,838.10,B,B2,Dealer,7 years,...,60.0,0.0,1.0,89700.0,41031.0,55300.0,34400.0,N,Cash,N
2,20000.0,20000.0,20000.0,36 months,9.44%,640.10,B,B1,Executive Assistant,< 1 year,...,33.3,0.0,0.0,67717.0,37895.0,22900.0,42517.0,N,Cash,N
3,10000.0,10000.0,10000.0,36 months,7.21%,309.74,A,A3,Regional Practice Director,7 years,...,40.0,0.0,0.0,330503.0,24170.0,46100.0,16613.0,N,Cash,N
4,15000.0,15000.0,15000.0,36 months,7.97%,469.84,A,A5,Senior Software Engineer,3 years,...,10.0,0.0,0.0,327003.0,51954.0,50000.0,39620.0,N,Cash,N


In [ ]:
# 再次用pandas解析预处理过的数据文件并预览基本信息。
loans = pd.read_csv('loans_2017q2.csv',encoding='utf-8')
print(loans.shape)
loans.head()

In [35]:
#数据从137列减少至102列。
loans.dtypes

loan_amnt                     float64
funded_amnt                   float64
funded_amnt_inv               float64
term                           object
int_rate                       object
installment                   float64
grade                          object
sub_grade                      object
emp_title                      object
emp_length                     object
home_ownership                 object
annual_inc                    float64
verification_status            object
issue_d                        object
loan_status                    object
pymnt_plan                     object
purpose                        object
title                          object
zip_code                       object
addr_state                     object
dti                           float64
delinq_2yrs                   float64
earliest_cr_line               object
inq_last_6mths                float64
mths_since_last_delinq        float64
open_acc                      float64
pub_rec     

In [37]:
loans.describe()

,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,105451.000000,105451.000000,105451.000000,105451.000000,1.054510e+05,105376.000000,105451.000000,105451.000000,54332.00000,105451.000000,...,105451.000000,105451.000000,105451.000000,104305.000000,105451.000000,105451.000000,1.054510e+05,1.054510e+05,105451.000000,1.054510e+05
mean,14589.070516,14589.070516,14585.195731,436.796015,8.045197e+04,18.987276,0.356052,0.509507,33.28613,11.849655,...,0.093380,2.073257,93.482781,40.556624,0.141089,0.075163,1.850722e+05,5.296400e+04,23659.481589,4.644022e+04
std,9478.834204,9478.834204,9476.119506,280.181663,6.822064e+04,14.145490,0.962047,0.809095,21.91821,5.776871,...,0.557542,1.838137,9.748996,36.008256,0.391480,0.441561,1.847481e+05,5.140751e+04,23191.726471,4.615661e+04
min,1000.000000,1000.000000,1000.000000,30.120000,0.000000e+00,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
25%,7000.000000,7000.000000,7000.000000,232.640000,4.800000e+04,12.230000,0.000000,0.000000,15.00000,8.000000,...,0.000000,1.000000,90.400000,0.000000,0.000000,0.000000,5.294800e+04,2.160150e+04,8500.000000,1.683250e+04
50%,12000.000000,12000.000000,12000.000000,361.380000,6.800000e+04,18.120000,0.000000,0.000000,30.00000,11.000000,...,0.000000,2.000000,97.400000,33.300000,0.000000,0.000000,1.197400e+05,3.898700e+04,16800.000000,3.500000e+04
75%,20000.000000,20000.000000,20000.000000,584.370000,9.700000e+04,24.580000,0.000000,1.000000,49.00000,15.000000,...,0.000000,3.000000,100.000000,66.700000,0.000000,0.000000,2.680685e+05,6.681950e+04,30900.000000,6.209950e+04
max,40000.000000,40000.000000,40000.000000,1719.830000,8.900000e+06,999.000000,42.000000,5.000000,161.00000,88.000000,...,42.000000,22.000000,100.000000,100.000000,7.000000,22.000000,4.460960e+06,1.310848e+06,407900.000000,1.349413e+06


Pandas的describe()不能统计数据类型为object的属性，部分数据int_rate和emp_length数据类型都是object，稍后分析数据时需将它们转化为类型为floate的数字类型。

数据集的属性较多，我们初步聚焦几个重要特征展开分析，特别是我们最关心的属性贷款状态。

In [38]:
used_col = ['loan_amnt', 'term', 'int_rate', 'grade', 
            'issue_d', 'addr_state', 'loan_status',
            'purpose', 'annual_inc', 'emp_length'] # 贷款金额、贷款期限、贷款利率、信用评级、业务发生时间、业务发生所在州、贷款状态、贷款用途
used_data = loans[used_col]
used_data.head(5)  # 数据预览

,loan_amnt,term,int_rate,grade,issue_d,addr_state,loan_status,purpose,annual_inc,emp_length
0,25000.0,60 months,15.05%,C,Jun-2017,AZ,Current,credit_card,60000.0,10+ years
1,26000.0,36 months,9.93%,B,Jun-2017,DE,Current,home_improvement,57900.0,7 years
2,20000.0,36 months,9.44%,B,Jun-2017,GA,Current,credit_card,70000.0,< 1 year
3,10000.0,36 months,7.21%,A,Jun-2017,FL,Current,credit_card,145000.0,7 years
4,15000.0,36 months,7.97%,A,Jun-2017,MO,Current,debt_consolidation,102000.0,3 years


In [39]:
column_null_count = used_data.apply(not_null_count)
print(column_null_count)

loan_amnt      2
term           2
int_rate       2
grade          2
issue_d        2
addr_state     2
loan_status    2
purpose        2
annual_inc     2
emp_length     2
dtype: int64


## 单变量分析
* 1.贷款状况分析

### 异常值处理

In [46]:
used_data['loan_status'].value_counts()

Current               93684
Fully Paid             7730
Late (31-120 days)     1711
In Grace Period        1394
Late (16-30 days)       473
Charged Off             458
Default                   1
Name: loan_status, dtype: int64

In [50]:
used_data[used_data['loan_status'] == 'Default']

,loan_amnt,term,int_rate,grade,issue_d,addr_state,loan_status,purpose,annual_inc,emp_length
3257,32000.0,36 months,9.44%,B,Jun-2017,CA,Default,home_improvement,125000.0,6 years


由于loan_status异常值为n的数量和贷款金额较小，因此我们直接删异常值所对应的行。

used_data = used_data.drop([6873,21814,51957]) 

为了更方便分析，我们将贷款状态进行分类变量编码，主要将贷款状态分为正常和违约，贷款状态分类依据主要参考[The 10 loan status variants explained](http://link.zhihu.com/?target=http%3A//help.bitbond.com/article/20-the-10-loan-status-variants-explained) 